In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../Chapter2/data_temp/raw/file0.csv")

In [1]:
import dask.dataframe as dd

In [4]:
ddf = dd.read_csv("../Chapter2/data_temp/raw/file0.csv")

In [5]:
ddf

,station,date,temperature_avg,extra
npartitions=1,,,,
,object,object,int64,float64
,...,...,...,...


In [6]:
df.dtypes

station             object
date                object
temperature_avg      int64
extra              float64
dtype: object

In [7]:
ddf.dtypes

station             object
date                object
temperature_avg      int64
extra              float64
dtype: object

In [8]:
df.head()

,station,date,temperature_avg,extra
0,A0,2020-01-01,17,1.0
1,A0,2020-01-02,20,1.0
2,A0,2020-01-03,19,1.0
3,A0,2020-01-04,17,1.0
4,A0,2020-01-05,12,1.0


In [9]:
ddf.head()

,station,date,temperature_avg,extra
0,A0,2020-01-01,17,1.0
1,A0,2020-01-02,20,1.0
2,A0,2020-01-03,19,1.0
3,A0,2020-01-04,17,1.0
4,A0,2020-01-05,12,1.0


In [10]:
df.tail()

,station,date,temperature_avg,extra
25,A0,2020-01-26,10,1.0
26,A0,2020-01-27,21,1.0
27,A0,2020-01-28,22,1.0
28,A0,2020-01-29,23,1.0
29,A0,2020-01-30,16,1.0


In [11]:
ddf.tail()

,station,date,temperature_avg,extra
25,A0,2020-01-26,10,1.0
26,A0,2020-01-27,21,1.0
27,A0,2020-01-28,22,1.0
28,A0,2020-01-29,23,1.0
29,A0,2020-01-30,16,1.0


In [12]:
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, station to extra
dtypes: object(2), float64(1), int64(1)

# Reading big file

In [1]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd

cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39471 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [2]:
fn = "../Chapter2/nyctaxi/raw/2016_Yellow_Taxi_Trip_Data.csv"
ddf = dd.read_csv(fn)

In [3]:
ddf

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
npartitions=271,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,float64,float64,int64,object,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
len(ddf)

131165043

In [8]:
ddf["trip_distance"].max().compute()

19072628.8

In [9]:
ddf.size

dd.Scalar<size-ag..., dtype=int64>

In [10]:
%%time
d = ddf["trip_distance"].describe().compute()

CPU times: user 1min 9s, sys: 4.71 s, total: 1min 14s
Wall time: 4min 30s


In [11]:
d

count    1.311650e+08
mean     4.768565e+00
std      3.819841e+03
min     -3.390584e+06
25%      1.000000e+00
50%      1.740000e+00
75%      3.300000e+00
max      1.907263e+07
Name: trip_distance, dtype: float64

In [3]:
import dask.dataframe as dd

In [12]:
path = "../Chapter2/nyctaxi/part_csv_stage1/*/*"
ddf = dd.read_csv(path)

In [67]:
path = "../Chapter2/nyctaxi/part_parquet_stage1/"
ddf = dd.read_parquet(path)

# To Do
* basic statistic as describe
* assigning a new column


In [1]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd

cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:44939 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [2]:
import dask.dataframe as dd

In [8]:
path = "../Chapter2/nyctaxi/part_parquet_stage1/"
ddf = dd.read_parquet(path)

In [17]:
import os

In [14]:
cols_to_rm = ['pickup_longitude',
              'pickup_latitude',
              'dropoff_longitude',
              'dropoff_latitude']
cols_to_read = [col for col in ddf.columns 
                if col not in cols_to_rm]

In [15]:
ddf = dd.read_parquet(path, columns=cols_to_read)

In [31]:
import os

def recursive_get_files(path):
    fns = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        if len(filenames) > 0:
            fns += [os.path.join(dirpath, fn) for fn in filenames]
    return fns

In [32]:
fns = recursive_get_files(path)

In [9]:
import pandas as pd


def extract_dtypes(df, name): 
    dtypes = df.dtypes.astype(str).to_frame(name=name) 
    return dtypes 


df_types = []
for year in [2016, 2017, 2018]:
    fldr = "../Chapter2/nyctaxi/raw"
    fn = f"{fldr}/{year}_Yellow_Taxi_Trip_Data.csv"
    df = dd.read_csv(fn)
    df_types.append(extract_dtypes(df, year))
df_types = pd.concat(df_types, axis=1)

In [10]:
df_types

,2016,2017,2018
VendorID,int64,int64,int64
tpep_pickup_datetime,object,object,object
tpep_dropoff_datetime,object,object,object
passenger_count,int64,int64,int64
trip_distance,float64,float64,float64
pickup_longitude,float64,NaN,NaN
pickup_latitude,float64,NaN,NaN
RatecodeID,int64,int64,int64
store_and_fwd_flag,object,object,object
dropoff_longitude,float64,NaN,NaN


In [11]:
mask = df_types.isnull().any(1)

In [12]:
cols_to_read = df_types[~mask].index

In [13]:
print(cols_to_read)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount', 'PULocationID',
       'DOLocationID'],
      dtype='object')


In [56]:
path = "../Chapter2/nyctaxi/part_parquet_stage1/"
ddf = dd.read_parquet(path, columns=cols_to_read)

In [2]:
ddf = []
for year in [2016, 2017, 2018]:
    path = f"../Chapter2/nyctaxi/part_parquet_stage1/year={year}/"
    ddf.append(dd.read_parquet(path))
ddf = dd.concat(ddf, ignore_index=True)

In [3]:
ddf["pickup_longitude"].mean().compute()

-72.92046880566828

In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:43243 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [6]:
ddf = []
for year in [2016, 2017, 2018]:
    path = f"../Chapter2/nyctaxi/part_parquet_stage1/year={year}/"
    ddf.append(dd.read_parquet(path))
ddf = dd.concat(ddf, ignore_index=True)

In [8]:
%%time
ddf.to_parquet("../Chapter2/nyctaxi/part_parquet_stage2/",
               write_index=False,
               partition_on=["year"])

ValueError: Partitioning on non-existent column. partition_on=['year'] .columns=['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'PULocationID', 'DOLocationID']

In [20]:
client.restart()

Client Scheduler: tcp://127.0.0.1:43243 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [15]:
path = "../Chapter2/nyctaxi/part_parquet_stage1/year=2018/"
ddf = dd.read_parquet(path, columns=cols_to_read)

In [18]:
ddf.sample(frac=0.01, random_state=0)
   .to_parquet("../Chapter2/nyctaxi/sample_parquet/")

In [28]:
%%time
ddf.sample(frac=0.01, random_state=0)\
   .to_parquet("../Chapter2/nyctaxi/sample_parquet_stage1/", write_index=False)

CPU times: user 12.5 s, sys: 1.03 s, total: 13.5 s
Wall time: 1min 9s


In [32]:
ddf1 = dd.read_parquet("../Chapter2/nyctaxi/sample_parquet_stage1/")

In [30]:
%%time
ddf1 = ddf1.repartition(npartitions=ddf.npartitions // 100)
ddf1.to_parquet("../Chapter2/nyctaxi/sample_parquet_stage2/", write_index=False)

CPU times: user 1.53 s, sys: 79.7 ms, total: 1.61 s
Wall time: 6.54 s


In [26]:
df =  pd.read_parquet("../Chapter2/nyctaxi/sample_parquet_stage2/")

In [31]:
ddf1 = ddf1.repartition(npartitions=1)
path_out = "../Chapter2/nyctaxi/sample.parquet"
ddf1.to_parquet(path_out, write_index=False)

In [33]:
ddf1 = dd.read_parquet("../Chapter2/nyctaxi/sample_parquet_stage1/")
fn = "../Chapter2/nyctaxi/sample.parquet"
ddf1.compute().to_parquet(fn, index=False)

In [34]:
ddf1 = dd.read_parquet("../Chapter2/nyctaxi/sample_parquet_stage1/")

In [36]:
path_out = "../Chapter2/nyctaxi/sample2.csv"
ddf1.to_csv(path_out, index=False, single_file=True)

['/media/data/Projects/Dask-Book/Machine-Learning-and-Data-Analysis-with-Dask/Chapter2/nyctaxi/sample2.csv']

# Partitions_on

In [3]:
path = "../Chapter2/nyctaxi/part_parquet_stage1/year=2018/"
ddf = dd.read_parquet(path)

In [5]:
o = ddf["VendorID"].nunique().compute()

In [6]:
o

3

In [7]:
ddf["VendorID"].unique().compute()

0    2
1    1
2    4
Name: VendorID, dtype: int64

In [8]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:39471 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


## pandas

In [9]:
import pandas as pd

In [11]:
path = "../Chapter2/nyctaxi/sample.parquet"
df = pd.read_parquet(path)

In [12]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
0,2,03/07/2018 04:31:50 PM,03/07/2018 04:36:28 PM,1,0.60,1,N,1,5.0,1.0,0.5,1.36,0.00,0.3,8.16,238,238
1,1,03/06/2018 08:22:40 PM,03/06/2018 08:29:35 PM,1,0.80,1,N,1,6.0,0.5,0.5,1.45,0.00,0.3,8.75,230,237
2,1,03/08/2018 02:49:03 AM,03/08/2018 03:06:11 AM,1,4.70,1,N,1,17.0,0.5,0.5,3.65,0.00,0.3,21.95,107,181
3,2,03/06/2018 10:41:25 AM,03/06/2018 11:17:00 AM,1,21.00,1,N,2,57.0,0.0,0.5,0.00,5.76,0.3,63.56,161,38
4,2,03/06/2018 05:12:48 PM,03/06/2018 05:19:50 PM,1,1.25,1,N,1,6.5,1.0,0.5,1.66,0.00,0.3,9.96,142,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122342,2,12/05/2018 12:07:39 AM,12/05/2018 12:21:37 AM,3,2.56,1,N,1,11.0,0.5,0.5,2.46,0.00,0.3,14.76,107,230
1122343,2,12/04/2018 10:32:52 PM,12/04/2018 10:33:00 PM,1,0.00,5,N,1,100.0,0.0,0.5,15.00,0.00,0.3,115.80,265,265
1122344,2,12/06/2018 11:08:08 AM,12/06/2018 11:17:13 AM,1,0.93,1,N,1,7.5,0.0,0.5,2.49,0.00,0.3,10.79,234,170
1122345,2,12/05/2018 08:19:18 AM,12/05/2018 08:28:42 AM,1,0.93,1,N,1,7.5,0.0,0.5,1.66,0.00,0.3,9.96,230,233


In [3]:
import pandas as pd
import os

path = "../Chapter2/nyctaxi/sample.parquet"
df = pd.read_parquet(path)

In [4]:
grps = [grp[1] for grp in list(df.groupby("VendorID"))]

In [6]:
grps = df.groupby("VendorID")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
1,1,03/06/2018 08:22:40 PM,03/06/2018 08:29:35 PM,1,0.8,1,N,1,6.0,0.5,0.5,1.45,0.0,0.3,8.75,230,237
2,1,03/08/2018 02:49:03 AM,03/08/2018 03:06:11 AM,1,4.7,1,N,1,17.0,0.5,0.5,3.65,0.0,0.3,21.95,107,181
5,1,03/05/2018 08:38:48 PM,03/05/2018 08:42:52 PM,1,0.5,1,N,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80,186,68
6,1,03/07/2018 12:26:37 PM,03/07/2018 12:31:16 PM,1,0.5,1,N,1,5.0,0.0,0.5,1.16,0.0,0.3,6.96,151,238
8,1,03/07/2018 02:01:08 PM,03/07/2018 02:05:20 PM,1,0.8,1,N,1,5.0,0.0,0.5,1.15,0.0,0.3,6.95,43,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122329,1,12/06/2018 10:08:45 AM,12/06/2018 10:22:10 AM,1,3.0,1,N,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55,79,256
1122336,1,12/05/2018 09:08:46 PM,12/05/2018 09:28:34 PM,1,2.6,1,N,1,14.0,0.5,0.5,3.06,0.0,0.3,18.36,162,90
1122337,1,12/04/2018 11:47:51 AM,12/04/2018 12:04:57 PM,1,1.9,1,N,1,11.5,0.0,0.5,1.50,0.0,0.3,13.80,142,236
1122340,1,12/06/2018 11:36:46 AM,12/06/2018 11:45:43 AM,1,0.8,1,N,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80,234,170


In [7]:
grp = grps[0]

In [11]:
vendor = grp["VendorID"].unique()[0]

In [16]:
fn = f"splitted_with_pandas/VendorId={vendor}/part1.csv"



In [17]:
os.makedirs(os.path.dirname(fn), exist_ok=True)

In [ ]:
grp.to_csv(fn)

In [18]:
def split_by_column(grp):
    vendor = grp["VendorID"].unique()[0]
    fn = f"splitted_with_pandas/VendorID={vendor}/part1.csv"
    os.makedirs(os.path.dirname(fn), exist_ok=True)
    grp.to_csv(fn)

In [21]:
def split_by_column_general(grp, column, fldr, index=False):
    name = grp[column].unique()[0]
    fn = f"{fldr}/{column}={name}/part1.csv"
    os.makedirs(os.path.dirname(fn), exist_ok=True)
    grp.to_csv(fn, index=index)

In [22]:
df.groupby("VendorID")\
  .apply(lambda x:
         split_by_column_general(x,
                                 "VendorID",
                                 "splitted_new",
                                 index=False))

""


In [29]:
def save_by_column_value(df, column, fldr, index=True):
    def split(grp, column, fldr, index=False):
        name = grp[column].unique()[0]
        fn = f"{fldr}/{column}={name}/part1.csv"
        os.makedirs(os.path.dirname(fn), exist_ok=True)
        grp.to_csv(fn, index=index)
    df.groupby(column)\
      .apply(lambda x: 
             split(x, column, fldr, index))

In [30]:
save_by_column_value(df, "VendorID", "splitted_new", False)

In [31]:
import dask.dataframe as dd

path = "../Chapter2/nyctaxi/sample.parquet"
ddf = dd.read_parquet(path)

In [32]:
ddf.to_parquet("splitted_with_dask",
               partition_on="VendorID",
               write_index=False)